In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px

In [ ]:
CITY="FB"

In [ ]:

weather_filename_DD = f"../data/weather/df_Dresden_weather_hourly 2025-03-28_20-51-37.pkl"
weather_filename_FB = f"../data/weather/df_Freiburg_weather_hourly 2025-03-28_20-51-37.pkl"
weather_df_DD = pd.read_pickle(weather_filename_DD)
weather_df_FB = pd.read_pickle(weather_filename_FB)
help_dict_weather = {'DD': weather_df_DD, 'FB': weather_df_FB}

In [ ]:
df_weather = help_dict_weather[CITY]
df_weather.index.name = "datetime_hour"

In [ ]:
datetime = "2025-03-19_10-47-56"
filename_DD = f"../data/nextbike/hourly_demand_supply_Dresden {datetime}.csv"
filename_FB = f"../data/nextbike/hourly_demand_supply_Freiburg {datetime}.csv"
df_trips_DD = pd.read_csv(filename_DD, index_col=None, parse_dates=["datetime_hour"])
df_trips_FB = pd.read_csv(filename_FB, index_col=None, parse_dates=["datetime_hour"])


help_dict = {'DD': df_trips_DD, 'FB': df_trips_FB}

In [ ]:
weather_df_FB.index.name = "datetime_hour"

In [ ]:
df_trips = help_dict[CITY]

In [ ]:
df_houly_trips= df_trips.drop(columns=["hex_id", "return_count"]).groupby("datetime_hour").agg("sum")

In [ ]:
# df_houly_trips

In [ ]:
df = df_houly_trips.copy()

In [ ]:
len(df)

In [ ]:
df_weather.loc[df.index]

In [ ]:
df = pd.concat([df, df_weather.loc[df.index]], axis=1)

In [ ]:
df.rename(columns={"rent_count": "Bike trips"}, inplace=True)

In [ ]:
df_total =  df.reset_index().resample("h", on="datetime_hour").agg({"Temperature": "mean", "Humidity": "mean", "Wind": "mean", "Precipitation": "sum", "Bike trips": "sum"})
df_total.dropna(subset="Temperature", inplace=True)

df_total.Temperature = (df_total.Temperature / 2).round() * 2
df_total.Temperature = df_total.Temperature.astype(int)
df_total.Humidity = df_total.Humidity.astype(int)


In [ ]:
df_total.Humidity=(df_total.Humidity / 5).round() * 5

In [ ]:
df_total['Rain']= df_total['Precipitation'].apply(lambda x: True if x > 0 else False)

In [ ]:
cols =  [col for col in df_total.columns if col != "Bike trips"]+["Bike trips"] 


df_total = df_total[cols]

In [ ]:
scatter_data=df_total.groupby(["Temperature", "Humidity"]).agg({"Bike trips": "mean"}).reset_index()

In [ ]:
fig = px.scatter(scatter_data, y="Humidity", x="Temperature", size="Bike trips", color="Bike trips", width=800, height=600)
fig.show()
fig.write_image(f'/Users/v.sinichenko/Downloads/plots/weather/scatter_humidity_temp_{CITY}.png', format='png')


In [ ]:
corr = df_total.corr()

In [ ]:
df_time_grouped =  df_total.reset_index().resample("4h", on="datetime_hour").agg({"Temperature": "mean", "Humidity": "mean", "Wind": "mean", "Precipitation": "sum", "Bike trips": "sum"})

In [ ]:
df_time_grouped

In [ ]:
df_time_grouped['Rain']= df_time_grouped['Precipitation'].apply(lambda x: True if x > 0 else False)

In [ ]:
df_total["Temperature"].describe()

In [ ]:
temp_bins = pd.IntervalIndex.from_tuples([(-20,-10), (-10,0), (0,10), (10,20), (20,30), (30,40)])
df_total["temp_bin"] = pd.cut(df_total["Temperature"], bins=temp_bins)

In [ ]:
# Create bins
df_total["temp_bin"] = pd.cut(df_total["Temperature"], bins=5, precision=0)
df_total["humid_bin"] = pd.cut(df_total["Humidity"], bins=5, precision=0)

# Pivot table
heatmap_data = df_total.pivot_table(index="humid_bin", columns="temp_bin", values="Bike trips", aggfunc="mean")

In [ ]:
heatmap_data

In [ ]:
heatmap_data = heatmap_data.sort_values(by="humid_bin", ascending=False)

In [ ]:
heatmap_data

In [ ]:
# plt.figure(figsize=(6, 5))
# cmap = sns.diverging_palette(220, 10, as_cmap=True)
# sns.heatmap(heatmap_data, cmap=cmap, annot=True, fmt=".0f")
# plt.xlabel("Temperature")
# plt.ylabel("Humidity")
# plt.savefig(f'/Users/v.sinichenko/Downloads/plots/weather/humidity_temp_heatmap_{CITY}.png', format='png', bbox_inches='tight')
# plt.show()


In [ ]:
df_time_grouped.dropna(subset=["Temperature", "Humidity", "Wind"], how="any", inplace=True)

In [ ]:
df_time_grouped.Temperature =df_time_grouped.Temperature.astype(int) 

In [ ]:
df_time_grouped.Humidity =df_time_grouped.Humidity.astype(int) 

In [ ]:
px.scatter(df_time_grouped, x="Temperature", y="Bike trips", color="Bike trips", size="Bike trips", width=800, height=600).show()

In [ ]:
px.scatter(df_time_grouped, x="Humidity", y="Bike trips", color="Bike trips", size="Bike trips", width=800, height=600).show()

In [ ]:
df_total

In [ ]:
fig = px.scatter(df_total, x="Temperature", y="Humidity", width=800, height=600)
fig.show()

In [ ]:
fig = px.scatter(df_total, x="Temperature", y="Humidity", color="Bike trips", size="Bike trips", width=800, height=600)
fig.show()

In [ ]:
fig = px.scatter(df_time_grouped, x="Temperature", y="Humidity", color="Bike trips", size="Bike trips", width=800, height=600)
fig.show()
fig.write_image(f'/Users/v.sinichenko/Downloads/plots/weather/scatter_humidity_temp_{CITY}', format='png')

In [ ]:
df_time_grouped[df_time_grouped.Temperature > 25]

In [ ]:
fig = px.scatter(df_time_grouped, x="Temperature", y="Humidity", color="Rain", size="Bike trips", width=800, height=600)
fig.show()
fig.write_image(f'/Users/v.sinichenko/Downloads/plots/weather/scatter_humidity_temp_rain_{CITY}.png', format='png')